In [ ]:
import sys
dir1 = '/Users/erihe/OneDrive - NTNU/'
if not dir1 in sys.path: sys.path.append(dir1)
from utils_new import *
    

## 94557

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '94557'
sessall = glob.glob(data_dir + '/' + mouse + '/*')
for sess in sessall[:1]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]
    sspk1 = sspk1[movetimes0,:]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[:,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[:,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2)

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.77
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:



num_neurons = len(sspk1[0,:])
rmap = np.zeros((num_neurons, 25, 25))
acorr = np.zeros((num_neurons, 25, 25))
numbins1 = 25
sig1 = 1

for i in range(num_neurons):
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                              sspk1[movetimes0,i], statistic='mean', 
                             bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
    rmap[i,:,:]  = mtot_tmp

hd_info = np.zeros(num_neurons)
for i in range(num_neurons):
    mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
    mu = np.mean(sspk1[:,i])
    hd_info[i] = information_score_1d(mtot, circ-1, mu)

scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
for i in rel_inds:
    mod_ind1s = np.where(ind1==i)[0]
    if len(mod_ind1s)>=2:
        print('Mod ', i)
        print('num_neurons ', len(mod_ind1s))
        print(mod_ind1s)
        sspk2 = sspk1[:,mod_ind1s]
        sspk2 = preprocessing.scale(sspk2[movetimes0])

        scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                       num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
        plt.show()




In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==10)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0))[0]
sspk2 = sspk2[movetimes0,:]

In [ ]:
n_points = 1200
dim = 6
eps = 0.2
k = 1000
spk1 = preprocessing.scale(sspk2,axis = 0)
dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(spk1, dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])                        
startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, metric = 'euclidean', 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1200,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             spk2 = sspk2,
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
plt.hsv()
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:

    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
plt.show()


In [ ]:
hd = headdirection[movetimes0]/360*2*np.pi


In [ ]:
cc = coords_mod1[:,1].copy()
dc = np.arctan2(np.mean(np.sin(cc- hd)), np.mean(np.cos(cc- hd)))%(2*np.pi)
cc -= dc 
cc = cc%(2*np.pi)
print(np.mean(np.abs(np.arctan2(np.sin(cc- hd), np.cos(cc- hd)))))

In [ ]:
plt.plot(cc[:1000])
plt.plot(hd[:1000])

In [ ]:
sessall

## 100072

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '100072'
sessall = glob.glob(data_dir + '/' + mouse + '/*')
for sess in sessall[:1]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2)

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.8
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:



num_neurons = len(sspk1[0,:])
rmap = np.zeros((num_neurons, 25, 25))
acorr = np.zeros((num_neurons, 25, 25))
numbins1 = 25
sig1 = 1

for i in range(num_neurons):
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[:,0], headpos[:,1],
                              sspk1[:,i], statistic='mean', 
                             bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
    rmap[i,:,:]  = mtot_tmp

hd_info = np.zeros(num_neurons)
for i in range(num_neurons):
    mtot, __, circ = binned_statistic(headdirection[:], sspk1[:,i], statistic = 'mean', bins = 30)
    mu = np.mean(sspk1[:,i])
    hd_info[i] = information_score_1d(mtot, circ-1, mu)

scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
for i in rel_inds:
    mod_ind1s = np.where(ind1==i)[0]
    if len(mod_ind1s)>=2:
        print('Mod ', i)
        print('num_neurons ', len(mod_ind1s))
        print(mod_ind1s)
        sspk2 = sspk1[:,mod_ind1s]
        sspk2 = preprocessing.scale(sspk2[movetimes0])

        scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                       num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
        plt.show()




In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==3)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1200
dim = 7
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, metric = 'euclidean', 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1200,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             spk2 = sspk2,
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
plt.hsv()
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:

    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
plt.show()


In [ ]:
hd = headdirection[movetimes0]/360*2*np.pi

In [ ]:
cc = coords_mod1[:,1].copy()
dc = np.arctan2(np.mean(np.sin(cc- hd)), np.mean(np.cos(cc- hd)))%(2*np.pi)
cc -= dc 
cc = cc%(2*np.pi)
print(np.mean(np.abs(np.arctan2(np.sin(cc- hd), np.cos(cc- hd)))))

In [ ]:
plt.plot(cc[:1000])
plt.plot(hd[:1000])

## 97045

## 20210305

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '97045'
sessall = glob.glob(data_dir + '/' + mouse + '/*')
for sess in sessall[:1]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.92
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:
    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 7
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:
for n_points in [1300,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


## 20210307

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '97045'
sessall = np.sort(glob.glob(data_dir + '/' + mouse + '/*'))
for sess in sessall[2:3]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]
    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False
    del filtered_events, nat_all, NeuronInformation
    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr1 = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr1,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:


import warnings
warnings.filterwarnings('ignore')
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
from torch.nn.functional import relu
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PairwiseDistance 
import torch.nn as nn
import torch.fft

from torch_topological.nn import SummaryStatisticLoss
from torch_topological.nn import VietorisRipsComplex

from tqdm import tqdm

import torch
import torch.optim as optim

class Ensembler(torch.nn.Module):
    def __init__(self, N, batch_size, maxdim, meshstart, meshstop, num_in_mesh,  
                 init_weights = [], seed=42, normalize_columns=False):
        super(Ensembler, self).__init__()
        self.normalize_columns = normalize_columns
        torch.manual_seed(seed)
        if len(init_weights) == 0:
            self.weights = torch.nn.Parameter(
              torch.randn(N, N),
              requires_grad=True)
        else:
            self.weights = torch.nn.Parameter(
              init_weights,
              requires_grad=True)

    def forward(self, x):        
        weights = torch.nn.functional.softmax(self.weights, dim=1)
        mask = (weights == weights.max(dim=0, keepdim=True)[0])
        weights = weights * mask
        out = torch.matmul(weights, x)
        return out, weights

torch.autograd.set_detect_anomaly(True)
class BettiCurves(nn.Module):
    """
    Produces the Betti curve of the diagram

    Parameters:

    meshstart: The lowest value at which to begin the curve
    meshstop: the highest value at which to stop the curve
    num_in_mesh: The number of evenly spaced points between meshstart and meshstop at which to compute the curve values

    Output:
    num_in_mesh dimensional vector of Betti curve values computed at num_in_mesh evenly spaced points starting at meshstart and ending at meshstop
    """
    def __init__(self, meshstart = 0, meshstop = 10, num_in_mesh = 1000, theta = 1000):
        super(BettiCurves, self).__init__()
        self.meshstart = meshstart
        self.meshstop = meshstop
        self.num_in_mesh = num_in_mesh
        self.theta = theta

    def forward(self, dgminfo):
        """ Betti PersLay """
        sp = torch.from_numpy(np.linspace(self.meshstart,self.meshstop,self.num_in_mesh)).repeat(len(dgminfo)).reshape(len(dgminfo),self.num_in_mesh).T
        return torch.sum(torch.sigmoid( 1e-6 + self.theta * (.5*(dgminfo[:,1]-dgminfo[:,0]) - torch.abs(sp - .5*(dgminfo[:,1]+dgminfo[:,0])))),1)



def from_ripser_to_giotto(dgm, infmax= np.inf):
    dgm_ret = []
    for dim, i in enumerate(dgm):
        i[np.isinf(i)] = infmax
        for j in i:
            dgm_ret.append([j[0],j[1], dim])
    return dgm_ret

class Teddy:
    def __init__(
            self,
            data_matrix,  # neurons x time steps
            maxdim=2,  # optimize the first signature[1] H_signature[0]s
            batch_size= 16,
            meshstart = 0, 
            meshstop = 100, 
            num_in_mesh = 1000,
            epochs=3,
            train_steps=10,  # per epoch
            lr=1e-2,
            seed=47,
            verbose=True,  # whether to print and plot intermediate statistics
            device="cpu",
            num_worse=5,  # how many epochs loss may get worse before stopping
            power=0.1,  # power in topological loss
            normalize_columns=False,
            save_file='teddy_weights',
            init_weights = [],
            numits = 1
    ):
        # Parameters
        self.T,self.N  = data_matrix.shape
        self.maxdim = maxdim
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_steps = train_steps
        self.lr = lr
        self.seed = seed
        self.verbose = verbose
        self.num_worse = num_worse
        self.power = power
        self.save_file = save_file
        self.betti_met = PairwiseDistance(metric='betti')
        self.meshstart = meshstart
        self.meshstop = meshstop
        self.num_in_mesh = num_in_mesh
        self.numits = numits
        # Setup Data, Model, Optimizer
        self.data_torch = torch.tensor(
            data_matrix, dtype=torch.float).to(device)
        self.rips = VietorisRipsComplex(dim=maxdim, p = 1).to(device)
        
        self.betti = BettiCurves(meshstart, meshstop, num_in_mesh).to(device)

        self.model = Ensembler(self.N, batch_size, maxdim, meshstart, 
                               meshstop, num_in_mesh,init_weights).to(device)
        self.optimizer = torch.optim.Adam([self.model.weights], lr=lr)
        self.weights = []
        self.Bs = []
        self.Bs1 = []


    def train(self):
        worse = 0  # counter for early stopping
        running_loss = 0.0
        running_loss_top = 0.0
        running_loss_rest = 0.0
        run_loss, run_weights = [], []
        run_Bs = []
        run_Bs1 = []
        t0 = time.time()
        self.stats = {'loss_top': [], 'loss_rest': [], 'loss': []}
        sp = torch.from_numpy(np.linspace(self.meshstart,self.meshstop,self.num_in_mesh))+1
        for i in range(self.epochs * self.train_steps + 1):
            # training step
            self.optimizer.zero_grad()
            Bs = torch.zeros(self.maxdim+1, self.num_in_mesh)
            Bs1 = torch.zeros(self.maxdim+1, self.num_in_mesh)
            l0 = torch.zeros(self.maxdim+1)
            l00 = torch.zeros(self.maxdim+1)
            for j in range(self.numits):
                np.random.seed(self.seed + i*self.numits + j)
                batch_ind = np.random.choice(self.T, self.batch_size, replace=False)
                out, weights = self.model(self.data_torch[batch_ind, :].T)
                out = torch.nn.functional.normalize(out,dim = 0, p = 1)
                #out = torchvision.transforms.Normalize(out)
                layer_out = self.rips(out.T + 1e-3)                  
                layer_out1 = self.rips(out + 1e-3)
                
                for d in range(self.maxdim+1):
                    lives0 = layer_out[d][1][:,1] - layer_out[d][1][:,0]
                    l0[d] += len(lives0)
                    lives00 = layer_out1[d][1][:,1] - layer_out1[d][1][:,0]
                    l00[d] += len(lives00)
                    if l0[d] > 0:
                        Bs1[d] += self.betti(layer_out[d][1][lives0>0,:])*sp*(l0[d]+l00[d])/(2*l0[d])#/max(1,len(lives0[lives0>0]))*sp
                    if l00[d] > 0:
                        Bs[d] += self.betti(layer_out1[d][1][lives00>0,:])*sp*(l0[d]+l00[d])/(2*l00[d])#/max(1,len(lives0[lives0>0]))*sp

#            loss = torch.sum(torch.pow((Bs-Bs1),2))                   
            print('loss', loss)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.lr)
            self.optimizer.step()

            # logging
            run_loss.append(loss.item())
            run_weights.append(self.model.weights.detach().cpu().numpy().copy())
            run_Bs.append([B0.cpu().detach().numpy() for B0 in Bs])
            run_Bs1.append([B0.cpu().detach().numpy() for B0 in Bs1])
            if i > -1 and not (i % self.train_steps):
                # early stopping
                run_best = np.argmin(run_loss)
                self.stats['loss'].append(run_loss[run_best])
                self.weights.append(run_weights[run_best])
                self.Bs.append(run_Bs[run_best])
                self.Bs1.append(run_Bs1[run_best])
                if self.verbose:
                    log = 'run=%s, time=%.2fs' % (i, time.time() - t0)
                    print(log)
                    self.plotting(run_weights[run_best], self.stats['loss'],  run_Bs[run_best], run_Bs1[run_best])
                if i > 0:
                    if self.stats['loss'][-1] > self.stats['loss'][-2]:
                        worse += 1
                        if worse > self.num_worse:
                            print('Early stopping at iteration', i)
                            break
                    else:
                        worse = 0 
                run_loss, run_weights, run_Bs, run_Bs1 = [], [], [], []
        print('Training finished.')


    def plotting(self, weights, run_loss, Bs, Bs1):
        xs = np.linspace(self.meshstart, self.meshstop, self.num_in_mesh)
        plt.figure()
        for d in range(self.maxdim+1):
            plt.plot(xs, Bs[d], c = ['k', 'r','g'][d])
        plt.figure()
        for d in range(self.maxdim+1):
            plt.plot(xs, Bs1[d], c = ['k', 'r','g'][d])
        
        plt.figure(figsize=(18, 3))        
        plt.subplot(1,3, 1)  # complete weight matrix
        plt.imshow(weights, vmin = np.percentile(weights.flatten(),2.5), 
                   vmax = np.percentile(weights.flatten(),97.5),)
        plt.colorbar()

        plt.subplot(1, 3, 2)  # max over columns of weight matrix
        ax1 = plt.gca()
        ax2 = ax1.twinx()
        ax1.plot(weights.max(0), 'g-')
        ax1.set_xlabel('weights')
        ax1.set_ylabel('max over columns', color='g', alpha=.5)
        
        plt.subplot(1, 3, 3)
        plt.plot(run_loss, '.-', label='loss')
        plt.title('loss')

        print(run_loss)
        plt.tight_layout()
        plt.show()


In [ ]:


import warnings
warnings.filterwarnings('ignore')
import torchvision
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
from torch.nn.functional import relu
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PairwiseDistance 
import torch.nn as nn
import torch.fft
from torch_topological.nn import SummaryStatisticLoss
from torch_topological.nn import VietorisRipsComplex

from tqdm import tqdm

import torch
import torch.optim as optim

class Ensembler(torch.nn.Module):
    def __init__(self, N, batch_size, maxdim, meshstart, meshstop, num_in_mesh,  
                 init_weights = [], seed=42, normalize_columns=False):
        super(Ensembler, self).__init__()
        self.normalize_columns = normalize_columns
        torch.manual_seed(seed)
        if len(init_weights) == 0:
            self.weights = torch.nn.Parameter(
              torch.randn(N, N),
              requires_grad=True)
        else:
            self.weights = torch.nn.Parameter(
              init_weights,
              requires_grad=True)

    def forward(self, x):        
        weights = torch.nn.functional.softmax(self.weights, dim=1)
        mask = (weights == weights.max(dim=0, keepdim=True)[0])
        weights = weights * mask
        out = torch.matmul(weights, x)
        return out, weights

torch.autograd.set_detect_anomaly(True)
class BettiCurves(nn.Module):
    """
    Produces the Betti curve of the diagram

    Parameters:

    meshstart: The lowest value at which to begin the curve
    meshstop: the highest value at which to stop the curve
    num_in_mesh: The number of evenly spaced points between meshstart and meshstop at which to compute the curve values

    Output:
    num_in_mesh dimensional vector of Betti curve values computed at num_in_mesh evenly spaced points starting at meshstart and ending at meshstop
    """
    def __init__(self, meshstart = 0, meshstop = 10, num_in_mesh = 1000, theta = 1000):
        super(BettiCurves, self).__init__()
        self.meshstart = meshstart
        self.meshstop = meshstop
        self.num_in_mesh = num_in_mesh
        self.theta = theta

    def forward(self, dgminfo):
        """ Betti PersLay """
        sp = torch.from_numpy(np.linspace(self.meshstart,self.meshstop,self.num_in_mesh)).repeat(len(dgminfo)).reshape(len(dgminfo),self.num_in_mesh).T
        return torch.sum(torch.sigmoid( 1e-6 + self.theta * (.5*(dgminfo[:,1]-dgminfo[:,0]) - torch.abs(sp - .5*(dgminfo[:,1]+dgminfo[:,0])))),1)



def from_ripser_to_giotto(dgm, infmax= np.inf):
    dgm_ret = []
    for dim, i in enumerate(dgm):
        i[np.isinf(i)] = infmax
        for j in i:
            dgm_ret.append([j[0],j[1], dim])
    return dgm_ret

class Teddy:
    def __init__(
            self,
            data_matrix,  # neurons x time steps
            maxdim=2,  # optimize the first signature[1] H_signature[0]s
            batch_size= 16,
            meshstart = 0, 
            meshstop = 100, 
            num_in_mesh = 1000,
            epochs=3,
            train_steps=10,  # per epoch
            lr=1e-2,
            seed=47,
            verbose=True,  # whether to print and plot intermediate statistics
            device="cpu",
            num_worse=5,  # how many epochs loss may get worse before stopping
            power=0.1,  # power in topological loss
            normalize_columns=False,
            save_file='teddy_weights',
            init_weights = [],
            numits = 1
    ):
        # Parameters
        self.T,self.N  = data_matrix.shape
        self.maxdim = maxdim
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_steps = train_steps
        self.lr = lr
        self.seed = seed
        self.verbose = verbose
        self.num_worse = num_worse
        self.power = power
        self.save_file = save_file
        self.betti_met = PairwiseDistance(metric='betti')
        self.meshstart = meshstart
        self.meshstop = meshstop
        self.num_in_mesh = num_in_mesh
        self.numits = numits
        # Setup Data, Model, Optimizer
        self.data_torch = torch.tensor(
            data_matrix, dtype=torch.float).to(device)
        self.rips = VietorisRipsComplex(dim=maxdim, p = 1).to(device)
        
        self.betti = BettiCurves(meshstart, meshstop, num_in_mesh).to(device)

        self.model = Ensembler(self.N, batch_size, maxdim, meshstart, 
                               meshstop, num_in_mesh,init_weights).to(device)
        self.optimizer = torch.optim.Adam([self.model.weights], lr=lr)
        self.weights = []
        self.Bs = []
        self.Bs1 = []


    def train(self):
        worse = 0  # counter for early stopping
        running_loss = 0.0
        running_loss_top = 0.0
        running_loss_rest = 0.0
        run_loss, run_weights = [], []
        run_Bs = []
        run_Bs1 = []
        t0 = time.time()
        self.stats = {'loss_top': [], 'loss_rest': [], 'loss': []}
        sp = torch.from_numpy(np.linspace(self.meshstart,self.meshstop,self.num_in_mesh))+1
        for i in range(self.epochs * self.train_steps + 1):
            # training step
            self.optimizer.zero_grad()
            Bs = torch.zeros(self.maxdim+1, self.num_in_mesh)
            Bs1 = torch.zeros(self.maxdim+1, self.num_in_mesh)
            l0 = torch.zeros(self.maxdim+1)
            l00 = torch.zeros(self.maxdim+1)
            for j in range(self.numits):
                np.random.seed(self.seed + i*self.numits + j)
                batch_ind = np.random.choice(self.T, self.batch_size, replace=False)
                out, weights = self.model(self.data_torch[batch_ind, :].T)
                out = torch.nn.functional.normalize(out,dim = 0, p =2 )
                layer_out = self.rips(out.T + 1e-3)                  
                
                for d in range(self.maxdim+1):
                    lives0 = layer_out[d][1][:,1] - layer_out[d][1][:,0]
                    l0[d] += len(lives0)
                    if l0[d] > 0:
                        Bs[d] += self.betti(layer_out[d][1][lives0>0,:])
            loss = -torch.sum(torch.diff(lives0)*lives0[1:])#/torch.sum(Bs)
            
            print('loss', loss)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.lr)
            self.optimizer.step()

            # logging
            run_loss.append(loss.item())
            run_weights.append(self.model.weights.detach().cpu().numpy().copy())
            run_Bs.append([B0.cpu().detach().numpy() for B0 in Bs])
            run_Bs1.append([B0.cpu().detach().numpy() for B0 in Bs1])
            if i > -1 and not (i % self.train_steps):
                # early stopping
                run_best = np.argmin(run_loss)
                self.stats['loss'].append(run_loss[run_best])
                self.weights.append(run_weights[run_best])
                self.Bs.append(run_Bs[run_best])
                self.Bs1.append(run_Bs1[run_best])
                if self.verbose:
                    log = 'run=%s, time=%.2fs' % (i, time.time() - t0)
                    print(log)
                    self.plotting(run_weights[run_best], self.stats['loss'],  run_Bs[run_best], run_Bs1[run_best])
                if i > 0:
                    if self.stats['loss'][-1] > self.stats['loss'][-2]:
                        worse += 1
                        if worse > self.num_worse:
                            print('Early stopping at iteration', i)
                            break
                    else:
                        worse = 0 
                run_loss, run_weights, run_Bs, run_Bs1 = [], [], [], []
        print('Training finished.')


    def plotting(self, weights, run_loss, Bs, Bs1):
        xs = np.linspace(self.meshstart, self.meshstop, self.num_in_mesh)
        plt.figure()
        for d in range(self.maxdim+1):
            plt.plot(xs, Bs[d], c = ['k', 'r','g'][d])
        plt.figure()
        for d in range(self.maxdim+1):
            plt.plot(xs, Bs1[d], c = ['k', 'r','g'][d])
        
        plt.figure(figsize=(18, 3))        
        plt.subplot(1,3, 1)  # complete weight matrix
        plt.imshow(weights, vmin = np.percentile(weights.flatten(),2.5), 
                   vmax = np.percentile(weights.flatten(),97.5),)
        plt.colorbar()

        plt.subplot(1, 3, 2)  # max over columns of weight matrix
        ax1 = plt.gca()
        ax2 = ax1.twinx()
        ax1.plot(weights.max(0), 'g-')
        ax1.set_xlabel('weights')
        ax1.set_ylabel('max over columns', color='g', alpha=.5)
        
        plt.subplot(1, 3, 3)
        plt.plot(run_loss, '.-', label='loss')
        plt.title('loss')

        print(run_loss)
        plt.tight_layout()
        plt.show()


In [ ]:
len(Xcorr1)

In [ ]:


import warnings
warnings.filterwarnings('ignore')
import torchvision
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
from torch.nn.functional import relu
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PairwiseDistance 
import torch.nn as nn
import torch.fft
from torch_topological.nn import SummaryStatisticLoss
from torch_topological.nn import VietorisRipsComplex

from tqdm import tqdm

import torch
import torch.optim as optim

class Ensembler(torch.nn.Module):
    def __init__(self, N, batch_size, maxdim, meshstart, meshstop, num_in_mesh,  
                 init_weights = [], seed=42, normalize_columns=False):
        super(Ensembler, self).__init__()
        self.normalize_columns = normalize_columns
        torch.manual_seed(seed)
        if len(init_weights) == 0:
            self.weights = torch.nn.Parameter(
              torch.randn(N, N),
              requires_grad=True)
        else:
            self.weights = torch.nn.Parameter(
              init_weights,
              requires_grad=True)

    def forward(self, x):        
        weights = torch.nn.functional.softmax(self.weights, dim=1)
        mask = (weights == weights.max(dim=0, keepdim=True)[0])
        weights = weights * mask
        out = torch.matmul(weights, x)
        return out, weights

torch.autograd.set_detect_anomaly(True)
class BettiCurves(nn.Module):
    """
    Produces the Betti curve of the diagram

    Parameters:

    meshstart: The lowest value at which to begin the curve
    meshstop: the highest value at which to stop the curve
    num_in_mesh: The number of evenly spaced points between meshstart and meshstop at which to compute the curve values

    Output:
    num_in_mesh dimensional vector of Betti curve values computed at num_in_mesh evenly spaced points starting at meshstart and ending at meshstop
    """
    def __init__(self, meshstart = 0, meshstop = 10, num_in_mesh = 1000, theta = 1000):
        super(BettiCurves, self).__init__()
        self.meshstart = meshstart
        self.meshstop = meshstop
        self.num_in_mesh = num_in_mesh
        self.theta = theta

    def forward(self, dgminfo):
        """ Betti PersLay """
        sp = torch.from_numpy(np.linspace(self.meshstart,self.meshstop,self.num_in_mesh)).repeat(len(dgminfo)).reshape(len(dgminfo),self.num_in_mesh).T
        return torch.sum(torch.sigmoid( 1e-6 + self.theta * (.5*(dgminfo[:,1]-dgminfo[:,0]) - torch.abs(sp - .5*(dgminfo[:,1]+dgminfo[:,0])))),1)



def from_ripser_to_giotto(dgm, infmax= np.inf):
    dgm_ret = []
    for dim, i in enumerate(dgm):
        i[np.isinf(i)] = infmax
        for j in i:
            dgm_ret.append([j[0],j[1], dim])
    return dgm_ret

class Teddy:
    def __init__(
            self,
            data_matrix,  # neurons x time steps
            maxdim=2,  # optimize the first signature[1] H_signature[0]s
            batch_size= 16,
            meshstart = 0, 
            meshstop = 100, 
            num_in_mesh = 1000,
            epochs=3,
            train_steps=10,  # per epoch
            lr=1e-2,
            seed=47,
            verbose=True,  # whether to print and plot intermediate statistics
            device="cpu",
            num_worse=5,  # how many epochs loss may get worse before stopping
            power=0.1,  # power in topological loss
            normalize_columns=False,
            save_file='teddy_weights',
            init_weights = [],
            numits = 1
    ):
        # Parameters
        self.T,self.N  = data_matrix.shape
        self.maxdim = maxdim
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_steps = train_steps
        self.lr = lr
        self.seed = seed
        self.verbose = verbose
        self.num_worse = num_worse
        self.power = power
        self.save_file = save_file
        self.betti_met = PairwiseDistance(metric='betti')
        self.meshstart = meshstart
        self.meshstop = meshstop
        self.num_in_mesh = num_in_mesh
        self.numits = numits
        # Setup Data, Model, Optimizer
        self.data_torch = torch.tensor(
            data_matrix, dtype=torch.float).to(device)
        self.rips = VietorisRipsComplex(dim=maxdim, p = 1).to(device)
        
        self.betti = BettiCurves(meshstart, meshstop, num_in_mesh).to(device)

        self.model = Ensembler(self.N, batch_size, maxdim, meshstart, 
                               meshstop, num_in_mesh,init_weights).to(device)
        self.optimizer = torch.optim.Adam([self.model.weights], lr=lr)
        self.weights = []
        self.Bs = []
        self.Bs1 = []


    def train(self):
        worse = 0  # counter for early stopping
        running_loss = 0.0
        running_loss_top = 0.0
        running_loss_rest = 0.0
        run_loss, run_weights = [], []
        run_Bs = []
        run_Bs1 = []
        t0 = time.time()
        self.stats = {'loss_top': [], 'loss_rest': [], 'loss': []}
        sp = torch.from_numpy(np.linspace(self.meshstart,self.meshstop,self.num_in_mesh))+1
        for i in range(self.epochs * self.train_steps + 1):
            # training step
            self.optimizer.zero_grad()
            Bs = torch.zeros(self.maxdim+1, self.num_in_mesh)
            Bs1 = torch.zeros(self.maxdim+1, self.num_in_mesh)
            l0 = torch.zeros(self.maxdim+1)
            l00 = torch.zeros(self.maxdim+1)
            for j in range(self.numits):
                np.random.seed(self.seed + i*self.numits + j)
                batch_ind = np.random.choice(self.T, self.batch_size, replace=False)
                out, weights = self.model(self.data_torch[batch_ind, :].T)
                print(out.shape)
                out = torch.nn.functional.normalize(out,dim = 0, p =2 ).T
                layer_out = self.rips(out + 1e-3)     
                for d in range(self.maxdim+1):
                    lives0 = layer_out[d][1][:,1] - layer_out[d][1][:,0]
                    l0[d] += len(lives0)
                    if l0[d] > 0:
                        Bs[d] += self.betti(layer_out[d][1][lives0>0,:])
                        if d == 0:
                            sortdist = torch.sort(torch.cdist(out, out),1).values
                            outfirst = torch.tensor(torch.concatenate((torch.zeros(len(out),1),
                                                                       sortdist[:, 1:2]),1))
                            Bs[d] -= self.betti(outfirst)

            #print(torch.sigmoid(torch.diff(lives0)))
#            print(lives0, torch.diff(lives0))
#            print(torch.mean(Bs*sp), torch.sum(torch.diff(lives0)*lives0[1:])*len(lives0))
#            loss = torch.mean(Bs*sp)-torch.sum(torch.diff(lives0)*lives0[1:])*len(lives0)
#            print(torch.diff(lives0))
#            loss = -torch.sum(torch.diff(lives0)*lives0[1:])/torch.sum(Bs)
            loss = -torch.sum(Bs)
            
            print('loss', loss)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.lr)
            self.optimizer.step()

            # logging
            run_loss.append(loss.item())
            run_weights.append(self.model.weights.detach().cpu().numpy().copy())
            run_Bs.append([B0.cpu().detach().numpy() for B0 in Bs])
            run_Bs1.append([B0.cpu().detach().numpy() for B0 in Bs1])
            if i > -1 and not (i % self.train_steps):
                # early stopping
                run_best = np.argmin(run_loss)
                self.stats['loss'].append(run_loss[run_best])
                self.weights.append(run_weights[run_best])
                self.Bs.append(run_Bs[run_best])
                self.Bs1.append(run_Bs1[run_best])
                if self.verbose:
                    log = 'run=%s, time=%.2fs' % (i, time.time() - t0)
                    print(log)
                    self.plotting(run_weights[run_best], self.stats['loss'],  run_Bs[run_best], run_Bs1[run_best])
                if i > 0:
                    if self.stats['loss'][-1] > self.stats['loss'][-2]:
                        worse += 1
                        if worse > self.num_worse:
                            print('Early stopping at iteration', i)
                            break
                    else:
                        worse = 0 
                run_loss, run_weights, run_Bs, run_Bs1 = [], [], [], []
        print('Training finished.')


    def plotting(self, weights, run_loss, Bs, Bs1):
        xs = np.linspace(self.meshstart, self.meshstop, self.num_in_mesh)
        plt.figure()
        for d in range(self.maxdim+1):
            plt.plot(xs, Bs[d], c = ['k', 'r','g'][d])
        plt.figure()
        for d in range(self.maxdim+1):
            plt.plot(xs, Bs1[d], c = ['k', 'r','g'][d])
        
        plt.figure(figsize=(18, 3))        
        plt.subplot(1,3, 1)  # complete weight matrix
        plt.imshow(weights, vmin = np.percentile(weights.flatten(),2.5), 
                   vmax = np.percentile(weights.flatten(),97.5),)
        plt.colorbar()

        plt.subplot(1, 3, 2)  # max over columns of weight matrix
        ax1 = plt.gca()
        ax2 = ax1.twinx()
        ax1.plot(weights.max(0), 'g-')
        ax1.set_xlabel('weights')
        ax1.set_ylabel('max over columns', color='g', alpha=.5)
        
        plt.subplot(1, 3, 3)
        plt.plot(run_loss, '.-', label='loss')
        plt.title('loss')

        print(run_loss)
        plt.tight_layout()
        plt.show()


In [ ]:
#xc = xc[:, indssort_bu<3000][:, indssort_bu1<2000]
model = Teddy( preprocessing.scale(sspk1,axis = 0),
            maxdim = 0,
            batch_size = 150,
            epochs = 100,
            train_steps = 100,
            meshstart = 0, 
            meshstop = 10, 
            num_in_mesh = 1000,            
            lr = 0.001,
            num_worse = 10,
            numits = 1,
#            init_weights = torch.tensor(model.weights[-1])
            )
model.train()
indssort = np.sum(model.weights,0)


In [ ]:
indssort = np.max(model.weights[np.argmin(model.stats['loss'])],0)
plt.plot(indssort)
#plt.plot(indssort1)
plt.plot(np.sort(indssort))

In [ ]:
indssort = np.sum(model.weights[np.argmin(model.stats['loss'])],0)
plt.plot(indssort)
#plt.plot(indssort1)
plt.plot(np.sort(indssort))

In [ ]:
#xtmp = Xcorr[:, indssort_bu<3000].copy()
#xtmp = xtmp[indssort_bu<3000, :]
#xtmp = xtmp[:, indssort_bu1<2000].copy()
#xtmp = xtmp[indssort_bu1<2000, :]
xtmp = Xcorr.copy() 

plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))
plt.show()
xtmp = xtmp[np.argsort(indssort),:]
xtmp = xtmp[:, np.argsort(indssort)]
plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))


In [ ]:
#xtmp = Xcorr1[:, indssort_bu<3000].copy()
#xtmp = xtmp[indssort_bu<3000, :]
#xtmp = xtmp[:, indssort_bu1<2000].copy()
#xtmp = xtmp[indssort_bu1<2000, :]
xtmp = Xcorr1.copy()
plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))
plt.show()
xtmp = xtmp[np.argsort(indssort),:]
xtmp = xtmp[:, np.argsort(indssort)]
plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))


In [ ]:
plt.scatter(hd_info, indssort)

In [ ]:
plt.scatter(np.sum(sspk1>0,0), indssort)

In [ ]:
for i in np.argsort(indssort)[:30]:
    plt.figure()
    plt.imshow(rmap[i])
    plt.show()

In [ ]:
for i in np.argsort(indssort)[-30:]:
    plt.figure()
    plt.imshow(rmap[i])
    plt.show()

In [ ]:
xtmp = Xcorr1.copy()

plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))
plt.show()
xtmp = xtmp[np.argsort(indssort),:]
xtmp = xtmp[:, np.argsort(indssort)]
plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))


In [ ]:
xtmp = Xcorr.copy()

plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))
plt.show()
xtmp = xtmp[np.argsort(indssort),:]
xtmp = xtmp[:, np.argsort(indssort)]
plt.imshow(xtmp, vmin = np.percentile(xtmp.flatten(), 2.5), vmax = np.percentile(xtmp.flatten(), 97.5))


In [ ]:
plt.scatter(hd_info, indssort)

In [ ]:
np.sum(indssort>3500)

In [ ]:
its = len(model.weights)
rangeits = np.arange(0,its,4)
plt.figure(figsize = (10,5), dpi = 120)
for i in rangeits:
    plt.plot(np.max(model.weights[i],0), label = str(i), c = cm.viridis(i/its))

plt.legend()

In [ ]:
its = len(model.Bs)
range_its = np.arange(its)
plt.figure(figsize = (10,5), dpi = 120)
for i in range_its:
    plt.plot(model.Bs[i][0], label = str(i), c = cm.viridis(i/its))
    print(i, np.sum(model.Bs[i][0]))
plt.legend()
plt.show()
plt.figure(figsize = (10,5), dpi = 120)
for i in range_its:
    plt.plot(model.Bs[i][1], label = str(i), c = cm.viridis(i/its))
    print(i, np.sum(model.Bs[i][1]))
plt.legend()
plt.show()

plt.figure(figsize = (10,5), dpi = 120)
for i in range_its:
    plt.plot(model.Bs[i][2], label = str(i), c = cm.viridis(i/its))
    print(i, np.sum(model.Bs[i][2]))
plt.legend()
plt.show()


In [ ]:
np.savez('Zong_data', sspikes_of1 = sspk2)

In [ ]:
    thr = 0.85
    ind1 = get_ind(Xcorr1,thr, linkage = 'average', bPlot = True)


In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 6
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [n_points,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:
coords_mod1 = get_coords_all(sspk2, 
                             coords_ds_consistent,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


In [ ]:
num_neurons = len(sspk2[0,:])

numfigs = 2
numw = 5
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 120)
plt.viridis()
nw = 0
sig1 = 1
numbins1 = 30

torsort = np.arange(num_neurons)#np.flip(np.argsort(pcorr))
cc = coords_mod1.copy()

for nn, n in enumerate(torsort):
    nnn = nn%numh
    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
        nw += 1
    num_neurons = len(sspk2[0,:])

    ax = plt.subplot(gs2[nnn,0]) 
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(xx1, yy1,
                                  sspk2[:,n], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    minval = 0 
    maxval = np.percentile(mtot_tmp.flatten(), 97.5)
    mtot_tmp[nans] = -np.inf
    ax.imshow(mtot_tmp, origin = 'lower', vmin = minval, vmax = maxval)
    ax.set_xticks([])
    ax.set_yticks([])


    ax = plt.subplot(gs2[nnn,1]) 
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(cc[:,0], cc[:,1],
                                          sspk2[:,n], statistic='mean', 
                                         bins=numbins1, range=None, expand_binnumbers=True)
    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    mtot_tmp = smooth_tuning_map(np.rot90(mtot_tmp,1), numbins1+1, sig1, bClose = True) 
    minval = 0 
    maxval = np.percentile(mtot_tmp.flatten(), 97.5)
    mtot_tmp[nans] = -np.inf
    ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], 
              vmin = minval, vmax = maxval)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1/ax.get_data_ratio())
    r_box = transforms.Affine2D().skew_deg(15,15)
    for x in ax.images + ax.lines + ax.collections:
        trans = x.get_transform()
        x.set_transform(r_box+trans) 
        if isinstance(x, PathCollection):
            transoff = x.get_offset_transform()
            x._transOffset = r_box+transoff     
    ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
    ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
    ax.set_aspect('equal', 'box') 
    ax.axis('off')   
plt.show()


## 20210308

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '97045'
sessall = glob.glob(data_dir + '/' + mouse + '/*')
for sess in sessall[3:4]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.92
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:
    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==1)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 6
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu,
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


## 20210314

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '97045'
sessall = glob.glob(data_dir + '/' + mouse + '/*')
for sess in sessall[4:5]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.92
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:
    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 4
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:
for n_points in [700,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


## 20210317

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '97045'
sessall = glob.glob(data_dir + '/' + mouse + '/*')
for sess in sessall[5:]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.87
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:
    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 6
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


## 97046

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse = '97046'
sessall = glob.glob(data_dir + '/' + mouse + '/*')


In [ ]:
sessall

## 20210307

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
for sess in sessall[:1]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
sessall

## 20210308

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
for sess in sessall[1:2]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds = np.where((bin_ind>=30) & (bin_ind<=650))[0]                             


In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==1)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 6
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


## 20210309

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
for sess in sessall[2:3]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 6
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


## 20210312

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
for sess in sessall[3:4]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.87
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:
    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==4)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 6
eps = 0.5
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


## 20210313

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
for sess in sessall[4:5]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.86
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:
    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 4
eps = 0.3
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1600,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:
for n_points in [800,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


In [ ]:
dim_red_spikes_move_scaled.shape

In [ ]:
nbs = 600 
metric = 'cosine'
X = squareform(pdist(dim_red_spikes_move_scaled[:,:], metric))
knn_indices = np.argsort(X)[:, :nbs]
knn_dists = X[np.arange(X.shape[0])[:, None], knn_indices].copy()
sigmas, rhos = smooth_knn_dist(knn_dists, nbs, local_connectivity=0)
rows, cols, vals = compute_membership_strengths(knn_indices, knn_dists, sigmas, rhos)
result = coo_matrix((vals, (rows, cols)), shape=(X.shape[0], X.shape[0]))
result.eliminate_zeros()
transpose = result.transpose()
prod_matrix = result.multiply(transpose)
result = (result + transpose - prod_matrix)
result.eliminate_zeros()
d = result.toarray()
d = -np.log(d)
np.fill_diagonal(d,0)
thresh = np.max(d[~np.isinf(d)])

In [ ]:
cpc = ComplexProjectiveCoords(d, n_landmarks= min(400, len(X)), distance_matrix=True)
fig = plt.figure()

plot_diagrams(cpc.dgms_, lifetime=True)


In [ ]:
coords = cpc.get_coordinates(perc=0.99, proj_dim=1, cocycle_idx=0, standard_range = False ,projective_dim_red_mode='one-by-one')
coords_R3 = ProjectiveMapUtils.hopf_map(coords)
coords_R2 = ProjectiveMapUtils.stereographic_projection_hemispheres(coords_R3)

In [ ]:

plt.axis("off")
_ = PlotUtils.plot_2sphere_boundary()
plt.scatter(coords_R2[:,0], coords_R2[:,1],c=dim_red_spikes_move_scaled[:,1])
_ = plt.axis("equal")

In [ ]:
plt.hsv()
plt.scatter(xx1[indstemp], yy1[indstemp], c = coords_R2[:,0])

In [ ]:
hd = headdirection[movetimes0]/360*2*np.pi


In [ ]:
cc = coords_mod1[:,0].copy()
dc = np.arctan2(np.mean(np.sin(cc- hd)), np.mean(np.cos(cc- hd)))%(2*np.pi)
cc -= dc 
cc = cc%(2*np.pi)
print(np.mean(np.abs(np.arctan2(np.sin(cc- hd), np.cos(cc- hd)))))

In [ ]:
plt.plot(cc[:1000])
plt.plot(hd[:1000])

## 20210313

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
for sess in sessall[5:]:
    print(sess)    
    NAT = h5py.File(sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]

    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    headdirection = headdirection[1:-1]
    speed = speed[1:-1]

    repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
    try:
        NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()][0,0])>0:
            repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    except:
        NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
        if len(NeuronInformation['RepeatCell'][()])>0:
            repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    indssort = np.where((spksum>0) & (spksum<10))[0]

    sig0  = 2
    sspk1 = sspikes[:,indssort]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
    sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

    lenc = 10                
    lenspk, num_neurons = np.shape(sspk1)           
    Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
    Xcorr[np.isnan(Xcorr)] = 1

    np.fill_diagonal(Xcorr,0)
    Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

    thr = 0.85
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


    movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]

    num_neurons = len(sspk1[0,:])
    rmap = np.zeros((num_neurons, 25, 25))
    acorr = np.zeros((num_neurons, 25, 25))
    numbins1 = 25
    sig1 = 1

    for i in range(num_neurons):
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[movetimes0,i], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


        nans = np.isnan(mtot_tmp)
        mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
        mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
        acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
        rmap[i,:,:]  = mtot_tmp

    hd_info = np.zeros(num_neurons)
    for i in range(num_neurons):
        mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[movetimes0,i], statistic = 'mean', bins = 30)
        mu = np.mean(sspk1[:,i])
        hd_info[i] = information_score_1d(mtot, circ-1, mu)

    scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
    thr = 0.83
    ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))


In [ ]:
    rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
    for i in rel_inds:
        mod_ind1s = np.where(ind1==i)[0]
        if len(mod_ind1s)>=2:
            print('Mod ', i)
            print('num_neurons ', len(mod_ind1s))
            print(mod_ind1s)
            sspk2 = sspk1[:,mod_ind1s]
            sspk2 = preprocessing.scale(sspk2[movetimes0])

            scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                           num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
            plt.show()









In [ ]:
rel_inds

In [ ]:
mod_inds = np.where(ind1==0)[0]
print(len(mod_inds))


In [ ]:
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0) & (speed>5))[0]
sspk2 = sspk2[movetimes0,:]
print(np.shape(sspk2))

In [ ]:
n_points = 1600
dim = 5
eps = 0.4
k = 1000

dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(preprocessing.scale(sspk2,axis = 0), dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])      

startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [1200,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:
for n_points in [800,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


In [ ]:

coords_mod1 = get_coords_all(sspk2, 
                             coords_ds,
                             np.arange(len(sspk2)),                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)


xx1 = headpos[movetimes0,0]
yy1 = headpos[movetimes0,1]
sig = 1
fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
fig2, axs2 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
for c in [0,1,]:
    plt.hsv()
    axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)
    axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    axs1[c].axis('off')
    

    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                              headpos[movetimes0,1][nans0],
                                              coords_mod1[:,c][nans0], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,sig)
    costot = gaussian_filter(costot,sig)
    mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    mtot[nans] = np.nan
    plt.viridis()
    axs2[c].imshow(mtot)
    axs2[c].axis('off')
    axs2[c].set_aspect(1/axs2[c].get_data_ratio())

    
plt.show()


In [ ]:
nbs = 600 
metric = 'euclidean'
X = squareform(pdist(dim_red_spikes_move_scaled[:,:], metric))
knn_indices = np.argsort(X)[:, :nbs]
knn_dists = X[np.arange(X.shape[0])[:, None], knn_indices].copy()
sigmas, rhos = smooth_knn_dist(knn_dists, nbs, local_connectivity=0)
rows, cols, vals = compute_membership_strengths(knn_indices, knn_dists, sigmas, rhos)
result = coo_matrix((vals, (rows, cols)), shape=(X.shape[0], X.shape[0]))
result.eliminate_zeros()
transpose = result.transpose()
prod_matrix = result.multiply(transpose)
result = (result + transpose - prod_matrix)
result.eliminate_zeros()
d = result.toarray()
d = -np.log(d)
np.fill_diagonal(d,0)
thresh = np.max(d[~np.isinf(d)])

In [ ]:
cpc = ComplexProjectiveCoords(d, n_landmarks= min(400, len(X)), distance_matrix=True)
fig = plt.figure()

plot_diagrams(cpc.dgms_, lifetime=True)


In [ ]:
coords = cpc.get_coordinates(perc=0.9, proj_dim=1, cocycle_idx=0, standard_range = False ,projective_dim_red_mode='one-by-one')
coords_R3 = ProjectiveMapUtils.hopf_map(coords)
coords_R2 = ProjectiveMapUtils.stereographic_projection_hemispheres(coords_R3)

In [ ]:

plt.axis("off")
_ = PlotUtils.plot_2sphere_boundary()
plt.scatter(coords_R2[:,0], coords_R2[:,1],c=dim_red_spikes_move_scaled[:,1])
_ = plt.axis("equal")

In [ ]:
plt.hsv()
plt.scatter(xx1[indstemp], yy1[indstemp], c = coords_R2[:,0])

In [ ]:
hd = headdirection[movetimes0]/360*2*np.pi


In [ ]:
cc = coords_mod1[:,0].copy()
dc = np.arctan2(np.mean(np.sin(cc- hd)), np.mean(np.cos(cc- hd)))%(2*np.pi)
cc -= dc 
cc = cc%(2*np.pi)
print(np.mean(np.abs(np.arctan2(np.sin(cc- hd), np.cos(cc- hd)))))

In [ ]:
plt.plot(cc[:1000])
plt.plot(hd[:1000])

In [ ]:
1

## Climb

In [ ]:
for mouse in mouse_all[-1:]:
    sessall = glob.glob(data_dir + '/' + mouse + '/*')
    for sess in sessall[:1]:
        print(sess)    
        NAT = h5py.File(sess + '/' + 'NAT.mat')
        nat_all = NAT[NAT['NAT'][()][0][0]][()]

        filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
        tt = nat_all[0,:]
        headpos = nat_all[1:3,:].T
        headdirection = nat_all[3,:]    
        speed = nat_all[4,:]

        sspikes = np.zeros(np.shape(filtered_events)).T
        for i in range(len(filtered_events)):
            if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
                continue
            if np.isnan(filtered_events[i,0]):
                sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
            else:
                sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
        sspikes = sspikes[1:-1,:]
        tt = tt[1:-1]
        headpos = headpos[1:-1,:]
        speed = speed[1:-1]

        repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
        try:
            NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
            if len(NeuronInformation['RepeatCell'][()][0,0])>0:
                repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
        except:
            NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
            if len(NeuronInformation['RepeatCell'][()])>0:
                repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

        sspikes = sspikes[:, repremove]
        sspikes[np.isnan(sspikes)] = 0
        sspikes[sspikes<0.001] = 0
        spksum = np.mean(sspikes,0)
        indssort = np.where((spksum>0) & (spksum<10))[0]

        sig0  = 2
        sspk1 = sspikes[:,indssort]
        sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
        sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

        lenc = 10                
        lenspk, num_neurons = np.shape(sspk1)           
        Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
        Xcorr[np.isnan(Xcorr)] = 1

        np.fill_diagonal(Xcorr,0)
        Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

        thr = 0.85
        ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
        bin_ind = np.bincount(ind1)
        numneuronsind = np.flip(np.argsort(bin_ind))


        movetimes0 = np.where((np.sum(sspk1,1)>0))[0]
        sspk1 = sspk1[movetimes0,:]

        num_neurons = len(sspk1[0,:])
        rmap = np.zeros((num_neurons, 25, 25))
        acorr = np.zeros((num_neurons, 25, 25))
        numbins1 = 25
        sig1 = 1

        for i in range(num_neurons):
            mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                      sspk1[:,i], statistic='mean', 
                                     bins=numbins1, range=None, expand_binnumbers=True)


            nans = np.isnan(mtot_tmp)
            mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
            mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
            acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
            rmap[i,:,:]  = mtot_tmp

        hd_info = np.zeros(num_neurons)
        for i in range(num_neurons):
            mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[:,i], statistic = 'mean', bins = 30)
            mu = np.mean(sspk1[:,i])
            hd_info[i] = information_score_1d(mtot, circ-1, mu)

        scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

        rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
        for i in rel_inds:
            mod_ind1s = np.where(ind1==i)[0]
            if len(mod_ind1s)>=2:
                print('Mod ', i)
                print('num_neurons ', len(mod_ind1s))
                print(mod_ind1s)
                sspk2 = sspk1[:,mod_ind1s]
                sspk2 = preprocessing.scale(sspk2)

                scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                               num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
                plt.show()









In [ ]:

        thr = 0.91
        ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
        bin_ind = np.bincount(ind1)
        numneuronsind = np.flip(np.argsort(bin_ind))



In [ ]:
headdirection = headdirection[1:-1]


In [ ]:

        sig0  = 2
        sspk1 = sspikes[:,indssort]
        spknull0 = sspk1.sum(0)>0
        sspk1 = sspk1[:, spknull0 ]
        sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
        sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))
        movetimes0 = np.where((np.sum(sspk1,1)>0))[0]
        sspk1 = sspk1[movetimes0,:]

In [ ]:

        num_neurons = len(sspk1[0,:])
        rmap = np.zeros((num_neurons, 25, 25))
        acorr = np.zeros((num_neurons, 25, 25))
        numbins1 = 25
        sig1 = 1

        for i in range(num_neurons):
            mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[:,0], headpos[:,1],
                                      sspk1[:,i], statistic='mean', 
                                     bins=numbins1, range=None, expand_binnumbers=True)


            nans = np.isnan(mtot_tmp)
            mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
            mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
            acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
            rmap[i,:,:]  = mtot_tmp

        hd_info = np.zeros(num_neurons)
        for i in range(num_neurons):
            mtot, __, circ = binned_statistic(headdirection[:], sspk1[:,i], statistic = 'mean', bins = 30)
            mu = np.mean(sspk1[:,i])
            hd_info[i] = information_score_1d(mtot, circ-1, mu)

        scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

        rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
        for i in rel_inds:
            mod_ind1s = np.where(ind1==i)[0]
            if len(mod_ind1s)>=2:
                print('Mod ', i)
                print('num_neurons ', len(mod_ind1s))
                print(mod_ind1s)
                sspk2 = sspk1[:,mod_ind1s]
                sspk2 = preprocessing.scale(sspk2)

                scores_cluster(sspk2, scores,  mod_ind1s,headpos[:,0], headpos[:, 1], spk2 = [], 
                               num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
                plt.show()






In [ ]:
mod_inds = np.where(ind1==3)[0]
print(len(mod_inds))
sig0  = 2
sspk2 = sspikes[:,indssort]
spknull0 = sspk2.sum(0)>0
sspk2 = sspk2[:, spknull0 ]
sspk2 = sspk2[:, mod_inds]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)

sspk2 = np.sqrt(gaussian_filter1d(sspk2, axis = 0, sigma = sig0))
movetimes0 = np.where((np.sum(sspk2,1)>0))[0]
sspk2 = sspk2[movetimes0,:]

In [ ]:
n_points = 1200
dim = 6
eps = 0.2
k = 1000
spk1 = preprocessing.scale(sspk2,axis = 0)
dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(spk1, dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])                        
startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, metric = 'euclidean', 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]


In [ ]:
for n_points in [700,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    dgms = persistence['dgms']
    fig = plot_barcode(dgms)
    
    
    
    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


# Visual cortex

In [ ]:
xy_all = {}
movetimes_all = {}
spk_all = {}
coords_all = {}
data_dir = 'weijan'
mouse_all = ['99725']
for mouse in mouse_all:
    sessall = glob.glob(data_dir + '/' + mouse + '/*')
    for sess in sessall:
        if (mouse == '97045') & (sess[-8:] in ('20210305','20210306','20210307', '20210308')):
                continue
        print(sess)    
        NAT = h5py.File(sess + '/' + 'NAT.mat')
        nat_all = NAT[NAT['NAT'][()][0][0]][()]

        filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
        tt = nat_all[0,:]
        headpos = nat_all[1:3,:].T
        headdirection = nat_all[3,:]    
        speed = nat_all[4,:]

        sspikes = np.zeros(np.shape(filtered_events)).T
        for i in range(len(filtered_events)):
            if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
                continue
            if np.isnan(filtered_events[i,0]):
                sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
            else:
                sspikes[1:-1,i] = scipy.interpolate.interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
        sspikes = sspikes[1:-1,:]
        tt = tt[1:-1]
        headpos = headpos[1:-1,:]
        speed = speed[1:-1]

        repremove = np.ones(len(sspikes[0,:]), dtype = bool)        
        try:
            NeuronInformation = sio.loadmat(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
            if len(NeuronInformation['RepeatCell'][()][0,0])>0:
                repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
        except:
            NeuronInformation = h5py.File(sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
            if len(NeuronInformation['RepeatCell'][()])>0:
                repremove[NeuronInformation['RepeatCell'][()][0].astype(int)-1] = False

        sspikes = sspikes[:, repremove]
        sspikes[np.isnan(sspikes)] = 0
        sspikes[sspikes<0.001] = 0
        spksum = np.mean(sspikes,0)
        indssort = np.where((spksum>0) & (spksum<10))[0]

        sig0  = 2
        sspk1 = sspikes[:,indssort]
        spknull0 = sspk1.sum(0)>0
        sspk1 = sspk1[:, spknull0 ]
        sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
        sspk1 = np.sqrt(gaussian_filter1d(sspk1, axis = 0, sigma = sig0))

        lenc = 10                
        lenspk, num_neurons = np.shape(sspk1)           
        Xcorr = cross_corr_dist(sspk1, lencorr = lenc)
        Xcorr[np.isnan(Xcorr)] = 1

        np.fill_diagonal(Xcorr,0)
        Xcorr = squareform(pdist(np.square(Xcorr), 'correlation'))

        thr = 0.8
        ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
        bin_ind = np.bincount(ind1)
        numneuronsind = np.flip(np.argsort(bin_ind))


        movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]
        sspk1 = sspk1[movetimes0,:]

        num_neurons = len(sspk1[0,:])
        rmap = np.zeros((num_neurons, 25, 25))
        acorr = np.zeros((num_neurons, 25, 25))
        numbins1 = 25
        sig1 = 1

        for i in range(num_neurons):
            mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                      sspk1[:,i], statistic='mean', 
                                     bins=numbins1, range=None, expand_binnumbers=True)


            nans = np.isnan(mtot_tmp)
            mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
            mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
            acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
            rmap[i,:,:]  = mtot_tmp

        hd_info = np.zeros(num_neurons)
        for i in range(num_neurons):
            mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[:,i], statistic = 'mean', bins = 30)
            mu = np.mean(sspk1[:,i])
            hd_info[i] = information_score_1d(mtot, circ-1, mu)

        scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

        rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
        for i in rel_inds:
            mod_ind1s = np.where(ind1==i)[0]
            if len(mod_ind1s)>=2:
                print('Mod ', i)
                print('num_neurons ', len(mod_ind1s))
                print(mod_ind1s)
                sspk2 = sspk1[:,mod_ind1s]
                sspk2 = preprocessing.scale(sspk2)

                scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                               num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
                plt.show()









In [ ]:
movetimes0 = np.where((speed>5) & (np.sum(sspk1,1)>0))[0]
sspk1 = sspk1[movetimes0,:]


In [ ]:

thr = 0.92
ind1 = get_ind(Xcorr,thr, linkage = 'average', bPlot = True)
bin_ind = np.bincount(ind1)
numneuronsind = np.flip(np.argsort(bin_ind))



In [ ]:


num_neurons = len(sspk1[0,:])
rmap = np.zeros((num_neurons, 25, 25))
acorr = np.zeros((num_neurons, 25, 25))
numbins1 = 25
sig1 = 1

for i in range(num_neurons):
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                              sspk1[:,i], statistic='mean', 
                             bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    acorr[i, :,:] = pearson_correlate2d(mtot_tmp, mtot_tmp)
    rmap[i,:,:]  = mtot_tmp

hd_info = np.zeros(num_neurons)
for i in range(num_neurons):
    mtot, __, circ = binned_statistic(headdirection[movetimes0], sspk1[:,i], statistic = 'mean', bins = 30)
    mu = np.mean(sspk1[:,i])
    hd_info[i] = information_score_1d(mtot, circ-1, mu)

scores = (('rmap', rmap), ('acorr2d', acorr), ('sum', np.sum(sspk1,0)), ('hd_info', hd_info))

rel_inds = np.where((bin_ind>=30) & (bin_ind<=500))[0]                             
for i in rel_inds:
    mod_ind1s = np.where(ind1==i)[0]
    if len(mod_ind1s)>=2:
        print('Mod ', i)
        print('num_neurons ', len(mod_ind1s))
        print(mod_ind1s)
        sspk2 = sspk1[:,mod_ind1s]
        sspk2 = preprocessing.scale(sspk2)

        scores_cluster(sspk2, scores,  mod_ind1s,headpos[movetimes0,0], headpos[movetimes0, 1], spk2 = [], 
                       num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
        plt.show()









In [ ]:
sspk2.shape

In [ ]:
n_points = 1200
dim = 5
eps = 0.2
k = 1000
spk1 = preprocessing.scale(sspk2,axis = 0)
dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(spk1, dim = dim)
dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])                        
startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, metric = 'euclidean', 
                                 epsilon = eps, startindex = startindex)
indstemp = []
if n_points > len(movetimes1):
    n_points = len(movetimes1)


In [ ]:
indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                   n_points, 1, metric = 'cosine')[0]
indstemp = movetimes1[indstemp]
for n_points in [1200,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    times  = movetimes0[indstemp].copy()
    dgms = persistence['dgms']


    coords_mod2 = coords_ds_consistent.T*2*np.pi

    inds, inds_label =  get_coord_distribution(coords_mod2, numbins = 50,epsilon = 0.1, metric = 'euclidean', startindex = -1)
    phases_1 = get_phases(sspk2[indstemp,:], coords_mod2, inds, inds_label)  
    pcorr1 = match_phases(coords_mod2, sspk2[indstemp,:], phases_1)

    # -ab
    coords_mod1 = coords_mod2.copy()
    coords_mod1[:,0] = 2*np.pi - coords_mod1[:,0]
    phases_2 = get_phases(sspk2[indstemp,:], coords_mod1, inds, inds_label)  
    pcorr2 = match_phases(coords_mod1, sspk2[indstemp,:], phases_2)
    if np.median(pcorr2)> np.median(pcorr1):
        coords_ds_consistent[:,0] = 1 - coords_ds_consistent[:,0]
        pcorr1 = pcorr2
        phases_1 = phases_2
    print(np.median(pcorr1))

    pcorrsqr = match_phases(coords_mod2, sspk2[indstemp,:], phases_1, bSqr = True)


    fig = plot_barcode(dgms)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())


    coords_mod1 = get_coords_all(sspk2, 
                                 coords_ds_consistent,
                                 np.arange(len(sspk2)),                             
                                 indstemp,
                                 dim = dim, 
                                 spk2 = sspk2,
                                 bPCA = True,
                                 bPred = False)

    for i in range(3):
        coords_mod1 += 1*i
        coords_mod1 = coords_mod1%(2*np.pi)
        fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
        for c in [0,1,]:

            mtot, __, __, circ  = binned_statistic_2d(xx1[times1][movetimes00],
                                                      yy1[times1][movetimes00],
                                                      coords_mod1[movetimes00,c], 
                                                      statistic = circmean, 
                                                      bins = 50,
                                                      expand_binnumbers = True)

            nans = np.isnan(mtot)
            sintot = np.sin(mtot)
            costot = np.cos(mtot)
            sintot[nans] = np.mean(sintot[~nans])
            costot[nans] = np.mean(costot[~nans])
            sintot = gaussian_filter(sintot,sig)
            costot = gaussian_filter(costot,sig)
            mtot = np.cos(np.arctan2(sintot, costot))
            plt.viridis()
            mtot[nans] = np.nan
            axs1[c].imshow(mtot)
            axs1[c].axis('off')
            axs1[c].set_aspect(1/axs1[c].get_data_ratio())

        plt.show()


In [ ]:
for n_points in [1000,]:
    print('n_points', n_points)
    if (len(indstemp)>0) & (n_points > len(indstemp)):
        continue
    indstemp = indstemp[:n_points]
    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
    thresh = np.max(d[~np.isinf(d)])    
    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
    print('movetimes1', len(movetimes1))
    n_points = len(indstemp)

    coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
    times  = movetimes0[indstemp].copy()
    dgms = persistence['dgms']


    coords_mod2 = coords_ds_consistent.T*2*np.pi

    inds, inds_label =  get_coord_distribution(coords_mod2, numbins = 50,epsilon = 0.1, metric = 'euclidean', startindex = -1)
    phases_1 = get_phases(sspk2[indstemp,:], coords_mod2, inds, inds_label)  
    pcorr1 = match_phases(coords_mod2, sspk2[indstemp,:], phases_1)

    # -ab
    coords_mod1 = coords_mod2.copy()
    coords_mod1[:,0] = 2*np.pi - coords_mod1[:,0]
    phases_2 = get_phases(sspk2[indstemp,:], coords_mod1, inds, inds_label)  
    pcorr2 = match_phases(coords_mod1, sspk2[indstemp,:], phases_2)
    if np.median(pcorr2)> np.median(pcorr1):
        coords_ds_consistent[:,0] = 1 - coords_ds_consistent[:,0]
        pcorr1 = pcorr2
        phases_1 = phases_2
    print(np.median(pcorr1))

    pcorrsqr = match_phases(coords_mod2, sspk2[indstemp,:], phases_1, bSqr = True)


    fig = plot_barcode(dgms)
    fig,ax = plt.subplots(1,1)
    ax.plot(var_exp[:15], lw = 2.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds)):
        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
    ax[2].scatter(*coords_ds[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
    for i in range(len(coords_ds_consistent)):
        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
    for i in range(3):
        ax[i].set_aspect(1/ax[i].get_data_ratio())

    coords_mod1 = get_coords_all(sspk2, 
                                 coords_ds,
                                 np.arange(len(sspk2)),                             
                                 indstemp,
                                 dim = dim, 
                                 spk2 = sspk2,
                                 bPCA = True,
                                 bPred = False)


    xx1 = headpos[movetimes0,0]
    yy1 = headpos[movetimes0,1]
    sig = 1
    plt.hsv()
    fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
    for c in [0,1,]:

        axs1[c].scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)

    plt.show()


In [ ]:
plt.hsv()
fig1, axs1 = plt.subplots(1,1, figsize = (10,10), dpi = 120)

for c in [0,]:

    axs1.scatter(xx1,yy1, c = coords_mod1[:,c], s = 5)

plt.show()


In [ ]:
plt.hsv()
plt.scatter(xx1,yy1, c = coords_mod1[:,0], s = 5)

In [ ]:
plt.plot(coords_mod1[:,0])

In [ ]:
spks = preprocessing.minmax_scale(sspikes,axis = 0)>0.2
for it, i in enumerate([11, 28, 183, 191, 126,133,120, 128, 140, 142,
                       2,4,  123,53,  21, 146, 178, 9, 17, 137,]):#np.unique(np.random.randint(len(dff[0,:]), size = 20)):

    plt.figure(figsize = (5,5), dpi = 120)
    plt.axis('off')
    plt.scatter(headpos[:,0],headpos[:,1], c = [[0.2,0.2,0.2]], alpha = 0.4, s = 2)
    plt.scatter(headpos[spks[:,i],0],headpos[spks[:,i],1], c = 'r', alpha = 0.7, s = 10)
    plt.savefig('Figs/' + mouse + '_' + sess + '/spk_map' + str(i) + '.png', transparent = True)
    plt.savefig('Figs/' + mouse + '_' + sess + '/spk_map' + str(i) + '.pdf', transparent = True)
    plt.show()

In [ ]:
mouse = '97045'
for sess in ['20210305', '20210307', '20210308', '20210317']:
    shuf_all = {}
    shuffle_name1 = 'weijan/' + mouse + '_' + sess + '_shuffles_concatenated.npz'
    shuffle_name2 = 'weijan/' + mouse + '_' + sess + '_shuffles_concatenated_2.npz'
    shuffle_name3 = 'weijan/' + mouse + '_' + sess + '_shuffles_concatenated3'
    shuf = np.load(shuffle_name1, allow_pickle = True)
    shuf1 = shuf['dgms_shuffles'][()]
    shuf.close()
    for s in shuf1:
        shuf_all[s] = shuf1[s]
        
    shuf = np.load(shuffle_name2, allow_pickle = True)
    shuf2 = shuf['dgms_shuffles'][()]
    shuf.close()
    for s in shuf2:
        shuf_all[s] = shuf2[s]
    np.savez(shuffle_name3, dgms_shuffles = shuf_all)
        
    
    f = np.load('weijan/' + mouse + '_' + sess + '_H2.npz', allow_pickle = True)
    dgms = list(f['dgms'])
    f.close()
    fig = plot_barcode(dgms, shuffle_name = shuffle_name3)
    fig.savefig('Figs/' + mouse + '_' + sess + '_barcode.png', transparent = True)
    fig.savefig('Figs/' + mouse + '_' + sess + '_barcode.pdf', transparent = True)


In [ ]:
coords_aligned = {}
for sess in sessall:
    coords_mod1 = coords_all[sess].copy()
    headpos = xy_all[sess].copy()
    movetimes0 = movetimes_all[sess].copy()
    if sess == '20210305':
        print('') 
    elif sess == '20210307':
        coords_mod1[:,0] += 2/np.sqrt(3)*coords_mod1[:,1]
    elif sess == '20210308':
        ctmp = coords_mod1[:,0].copy()
        coords_mod1[:,0] = 2*np.pi-coords_mod1[:,1]
        coords_mod1[:,1] = ctmp.copy()+ 2/np.sqrt(3)*coords_mod1[:,0]
    elif sess == '20210317':
        coords_mod1[:,1] += 2/np.sqrt(3)*coords_mod1[:,0]
    coords_mod1 = coords_mod1%(2*np.pi)
    coords_aligned[sess] = coords_mod1.copy()
    for c in [0,1,]:
        fig, axs = plt.subplots(1,1)
        nans0 = ~np.isnan(coords_mod1[:,c])
        mtot, __, __, circ  = binned_statistic_2d(headpos[movetimes0,0][nans0],
                                                  headpos[movetimes0,1][nans0],
                                                  coords_mod1[:,c][nans0], 
                                                  statistic = circmean, 
                                                  bins = 50,
                                                  expand_binnumbers = True)

        nans = np.isnan(mtot)
        sintot = np.sin(mtot)
        costot = np.cos(mtot)
        sintot[nans] = np.mean(sintot[~nans])
        costot[nans] = np.mean(costot[~nans])
        sintot = gaussian_filter(sintot,sig)
        costot = gaussian_filter(costot,sig)
        mtot = np.cos(np.arctan2(sintot, costot))
#            mtot = gaussian_filter(mtot, 1)
        plt.viridis()
        mtot[nans] = np.nan
        axs.imshow(mtot)
        axs.axis('off')
        axs.set_aspect(1/axs.get_data_ratio())
        fig.savefig('Figs/' + mouse + '_' + sess + '_stripes' + str(c) + '.png', transparent = True)
        fig.savefig('Figs/' + mouse + '_' + sess + '_stripes' + str(c) + '.pdf', transparent = True)
    plt.show()


In [ ]:
for sess in sessall:
    coords_mod1 = coords_all[sess].copy()
    headpos = xy_all[sess].copy()
    movetimes0 = movetimes_all[sess].copy()
    coords_mod1 = coords_aligned[sess].copy()

    im = plt.imread('C:\\Users\\Finnern\\OneDrive - NTNU\\Giocomo\\image044.png')
    im = np.array(im)
    a1 = np.rot90(im, 1)
    sp = -np.inf
    sig = 1

    cc = np.arctan2(gaussian_filter1d(np.sin(coords_mod1),sigma = sig,axis = 0),
                   gaussian_filter1d(np.cos(coords_mod1),sigma = sig,axis = 0))%(2*np.pi)
    bCos = True
    if bCos:
        eps = 0.0001 
        digitized = np.concatenate((np.digitize(np.cos(cc[:, 0]), np.linspace(-1-eps,1+eps, len(a1)+1))[:,np.newaxis], 
                            np.digitize(np.cos(cc[:, 1]), np.linspace(-1-eps,1+eps, len(a1)+1))[:,np.newaxis]),1)
    else:
        digitized = np.concatenate((np.digitize(cc[:, 0], np.linspace(0,2*np.pi, len(a1)+1))[:,np.newaxis], 
                                   np.digitize(cc[:, 1], np.linspace(0,2*np.pi, len(a1)+1))[:,np.newaxis]),1)
    cc1 = []
    for i in range(len(digitized)):
        cc1.append(a1[digitized[i,1]-1, digitized[i,0]-1]) 
    fig = plt.figure(figsize = (10,5), dpi = 200)
    plt.axis('off')
    plt.hsv()
    ax1 = fig.add_subplot(111)
    ax1.axis('off')
    im = ax1.scatter(headpos[movetimes0,0], headpos[movetimes0,1], s = 7, c = cc1, alpha  =0.7)
    ax1.set_aspect(1/ax1.get_data_ratio())
#    plt.show()
    fig.savefig('Figs/' + mouse + '_' + sess + '_stripes' + str(c) + '_2dcoords.png', transparent = True)
    fig.savefig('Figs/' + mouse + '_' + sess + '_stripes' + str(c) + '_2dcoords.pdf', transparent = True)




In [ ]:


f = np.load('weijan\\97045_20210308_glm.npz', allow_pickle = True)
tor3 = f['tor3'][()][:,0,0]
space3 = f['space3'][()][:,0,0]
f.close()                

from matplotlib.collections import PathCollection
import cv2 as cv
numbins1 = 30
sig1 = 2
roll = 3
plt.viridis()
for sessit, sess in enumerate(sessall):
    try:
        os.mkdir('Figs/' + mouse + '_' + sess )
    except:
        print('folder made')
        
    NAT = h5py.File(data_dir + '/' + mouse + '/' + sess + '/' + 'NAT.mat')
    nat_all = NAT[NAT['NAT'][()][0][0]][()]
    filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
    tt = nat_all[0,:]
    headpos = nat_all[1:3,:].T
    headdirection = nat_all[3,:]    
    speed = nat_all[4,:]

    sspikes = np.zeros(np.shape(filtered_events)).T
    for i in range(len(filtered_events)):
        if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
            continue
        if np.isnan(filtered_events[i,0]):
            sspikes[1:-1,i] = interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
        else:
            sspikes[1:-1,i] = interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    
    
        
    sspikes = sspikes[1:-1,:]
    tt = tt[1:-1]
    headpos = headpos[1:-1,:]
    speed = speed[1:-1]
    
    NeuronInformation = sio.loadmat(data_dir + '/' + mouse + '/' + sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
    repremove = np.ones(len(sspikes[0,:]), dtype = bool)
    repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
    sspikes = sspikes[:, repremove]
    sspikes[np.isnan(sspikes)] = 0
    sspikes[sspikes<0.001] = 0
    spksum = np.mean(sspikes,0)
    plt.plot(np.sort(spksum))
    indssort = np.where((spksum>0) & (spksum<10))   [0]
    
    movetimes0 = np.where(speed>5)[0]
    sspk1 = sspikes[:,indssort][movetimes0,:]#,sigma = 1, axis = 0)[:,indssort][movetimes0,:]
    spknull0 = sspk1.sum(0)>0
    sspk1 = sspk1[:, spknull0 ]
    spknull = np.sum(sspk1,1)>0
    sspk1 = sspk1[spknull,:]
    movetimes0 = movetimes0[spknull]
    
    xy_all[sess] = headpos.copy()
    movetimes_all[sess] = movetimes0.copy()
            
    num_neurons = len(sspk1[0,:])
    pp = coords_aligned[sess].copy()%(2*np.pi)

    
    numfigs = 2
    numw = 8
    numh = int(np.ceil(num_neurons/numw))
    outer1 = gridspec.GridSpec(1, numw)
    fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)))
    len_acorr = 500 #len(acorr_sess[fi][0,:])
    nw = 0
    torsort = np.flip(np.argsort(tor3))

    
    

In [ ]:

tor3[np.isnan(tor3)] = 0
space3[np.isnan(space3)] = 0
numfigs = 2
numw = 12
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)))
len_acorr = 500 #len(acorr_sess[fi][0,:])
nw = 0
torsort = np.flip(np.argsort(tor3))
for nn, n in enumerate(range(num_neurons)):
    nnn = nn%numh

    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
        nw += 1
    xnum = 0
    spk = sspk1[:,torsort[n]].copy()

    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  spk, statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    mtot_tmp[nans] = np.nan

    ax = plt.subplot(gs2[nnn,xnum]) 
#        ax.set_title(str(np.round(tor3[torsort[n]],2)) + ' | '
#                     + str(np.round(space3[torsort[n]],2)), fontdict = {'fontsize':10}, pad = 0.25)
    xnum += 1
    maxtot = np.sort(mtot_tmp.flatten())
    maxtot = maxtot[int(0.975*len(maxtot))]
    ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = 0, vmax = maxtot)
    ax.set_xticks([])
    ax.set_yticks([])


    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(pp[:,0], pp[:,1],
                                          spk, statistic='mean', 
                                         bins=numbins1, range=None, expand_binnumbers=True)
    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    mtot_tmp = smooth_tuning_map(np.rot90(mtot_tmp,1), numbins1+1, sig1, bClose = True) 
    mtot_tmp[nans] = -np.inf
    ax = plt.subplot(gs2[nnn,xnum]) 
    xnum += 1
    maxtot = np.sort(mtot_tmp.flatten())
    maxtot = maxtot[int(0.975*len(maxtot))]
    ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = 0, vmax = maxtot)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1/ax.get_data_ratio())
    r_box = transforms.Affine2D().skew_deg(15,15)

    for x in ax.images + ax.lines + ax.collections:
        trans = x.get_transform()
        x.set_transform(r_box+trans) 
        if isinstance(x, PathCollection):
            transoff = x.get_offset_transform()
            x._transOffset = r_box+transoff     
    ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
    ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
    ax.set_aspect('equal', 'box') 
    ax.axis('off')   
fig.savefig('Figs/' + mouse +'_' + sess+ '/torall.png', transparent = True)
fig.savefig('Figs/' + mouse +'_' + sess+ '/torall.pdf', transparent = True)
plt.close()


In [ ]:
for sess in sessall[2:3]:
    f = np.load(data_dir + '/' + mouse + '/' + sess + '/decoding_data.npz')
    indstemp = f['indstemp']
    coords_ds = f['coords_ds']
    f.close()
    sspk1 = np.sqrt(spk_all[sess])    
    coords_mod1 = coords_ds.T*2*np.pi
    headpos = xy_all[sess].copy()
    movetimes0 = movetimes_all[sess].copy()
    if sess == '20210305':
        print('') 
    elif sess == '20210307':
        coords_mod1[:,0] += 2/np.sqrt(3)*coords_mod1[:,1]
    elif sess == '20210308':
        ctmp = coords_mod1[:,0].copy()
        coords_mod1[:,0] = 2*np.pi-coords_mod1[:,1]
        coords_mod1[:,1] = ctmp.copy()+ 2/np.sqrt(3)*coords_mod1[:,0]
    elif sess == '20210317':
        coords_mod1[:,1] += 2/np.sqrt(3)*coords_mod1[:,0]
    coords_mod1 = coords_mod1%(2*np.pi)
    coords_ds = coords_mod1.T/(2*np.pi)
    coords_ds[0,:] = 1-coords_ds[0,:]
    starttime = 1000
    num_frames = 1000
    coordsnew_mod22 = get_coords_distribution1(sspk1, 
                                           sspk1, 
                                           coords_ds, indstemp, 
                                           starttime = starttime, num_frames = num_frames)


In [ ]:
from matplotlib import animation, cm, transforms, pyplot as plt
from IPython.display import HTML
from matplotlib.collections import PathCollection
headpos1 = np.flip(preprocessing.minmax_scale(headpos,axis = 0)[movetimes0],1)
headpos1[:,0] = 1-headpos1[:,0]
fig = plt.figure()
ax = fig.add_subplot(121)
ax.axis('off') 
ax.set_aspect('equal','box')
img = ax.scatter([], [], zorder = -1)
ax.set_ylim( 0, 1)
ax.set_xlim( 0, 1)

ax2 = fig.add_subplot(122)
ax2.set_ylim( 0, 2*np.pi+3/5*np.pi)
ax2.set_xlim( 0, 2*np.pi+3/5*np.pi)
ax2.set_aspect('equal','box')
ax2.axis('off')

img2 = ax2.imshow(coordsnew_mod22[0,:,:], origin = 'lower',
                  extent = [0,2*np.pi, 0, 2*np.pi], vmin = 0, vmax = 20)
ax2.text(0.7,-0.65-0.35, '0$\degree$')
ax2.text(2*np.pi-0.6,-0.65-0.35, '360$\degree$')
ax2.text(-0.6-0.45,0.6, '0$\degree$')
ax2.text(-0.85-0.65,2*np.pi-0.35,'360$\degree$')

im = [img, img2,]
def init():
    im[0].set_offsets(np.array([[],[]]).T)
    im[1].set_data([[]])
    return im

r_box = transforms.Affine2D().skew_deg(15,15)
for x in (ax2.images + ax2.lines + ax2.collections + ax2.texts):
    trans = x.get_transform()
    x.set_transform(r_box+trans) 
    if isinstance(x, PathCollection):
        transoff = x.get_offset_transform()
        x._transOffset = r_box+transoff     


fps = 25
height, width = 1600, 1600

def animate(i):
    i+= starttime    
    im[0].set_offsets(headpos1[starttime:i+1,:])
    im[1].set_data(np.rot90(coordsnew_mod22[i-starttime,:,:],3))    
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=1000, interval=100, blit=True)
HTML(anim.to_html5_video())

## Plot ratemaps according to GLM score

In [ ]:
f = np.load('weijan/' + mouse + '_' + sess + '_glm.npz', allow_pickle = True)

argwhere = np.argsort(tor3)
#argwhere = argwhere[(np.linspace(0, 1, 17)[np.arange(1,16,2)]*len(tor3)).astype(int)]
argwhere = argwhere[np.round(np.linspace(0, 1, 11)[np.arange(1,11,2)]*len(tor3)).astype(int)]
from matplotlib.collections import PathCollection
import cv2 as cv
numbins1 = 30
sig1 = 2
roll = 3

plt.viridis()

try:
    os.mkdir('Figs/' + mouse + '_' + sess )
except:
    print('folder made')

NAT = h5py.File(data_dir + '/' + mouse + '/' + sess + '/' + 'NAT.mat')
nat_all = NAT[NAT['NAT'][()][0][0]][()]
filtered_events = nat_all[np.arange(15,len(nat_all), 4),:]
tt = nat_all[0,:]
headpos = nat_all[1:3,:].T
headdirection = nat_all[3,:]    
speed = nat_all[4,:]

sspikes = np.zeros(np.shape(filtered_events)).T
for i in range(len(filtered_events)):
    if np.sum(np.isnan(filtered_events[i,:]))== len(tt):
        continue
    if np.isnan(filtered_events[i,0]):
        sspikes[1:-1,i] = interp1d(tt[np.arange(1,len(tt),2)], filtered_events[i,np.arange(1,len(tt),2)])(tt[1:-1])
    else:
        sspikes[1:-1,i] = interp1d(tt[np.arange(0,len(tt),2)], filtered_events[i,np.arange(0,len(tt),2)])(tt[1:-1])    


sspikes = sspikes[1:-1,:]
tt = tt[1:-1]
headpos = headpos[1:-1,:]
speed = speed[1:-1]

NeuronInformation = sio.loadmat(data_dir + '/' + mouse + '/' + sess + '/' + 'NeuronInformation.mat')['NeuronInformation']
repremove = np.ones(len(sspikes[0,:]), dtype = bool)
repremove[NeuronInformation['RepeatCell'][()][0,0][0].astype(int)-1] = False
sspikes = sspikes[:, repremove]
sspikes[np.isnan(sspikes)] = 0
sspikes[sspikes<0.001] = 0
spksum = np.mean(sspikes,0)
plt.plot(np.sort(spksum))
indssort = np.where((spksum>0) & (spksum<10))   [0]

movetimes0 = np.where(speed>5)[0]
sspk1 = sspikes[:,indssort][movetimes0,:]#,sigma = 1, axis = 0)[:,indssort][movetimes0,:]
spknull0 = sspk1.sum(0)>0
sspk1 = sspk1[:, spknull0 ]
spknull = np.sum(sspk1,1)>0
sspk1 = sspk1[spknull,:]
movetimes0 = movetimes0[spknull]

xy_all[sess] = headpos.copy()
movetimes_all[sess] = movetimes0.copy()

num_neurons = len(sspk1[0,:])
pp = coords_all[sess].copy()%(2*np.pi)


for it,  n in enumerate(argwhere):#enumerate(IsGridCell[:5].astype(int)):    
    print(tor3[n], space3[n])
    fig, axs = plt.subplots(1,2)
    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(headpos[movetimes0,0], headpos[movetimes0,1],
                                  sspk1[:,n], statistic='mean', 
                                 bins=numbins1, range=None, expand_binnumbers=True)


    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    nans = np.isnan(mtot_tmp)
    mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
    mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
    mtot_tmp[nans] = np.nan

    ax = axs[0]
    ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = 0, vmax = np.max(mtot_tmp) *0.975)
    ax.set_xticks([])
    ax.set_yticks([])

    mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(pp[:,0], pp[:,1],
                                          sspk1[:,n], statistic='mean', 
                                         bins=numbins1, range=None, expand_binnumbers=True)
    nans = np.isnan(mtot_tmp)
    mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
    mtot_tmp = smooth_tuning_map(np.rot90(mtot_tmp,1), numbins1+1, sig1, bClose = True) 
    mtot_tmp[nans] = -np.inf
    ax = axs[1]
    ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = 0, vmax = np.max(mtot_tmp) *0.975)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1/ax.get_data_ratio())
    r_box = transforms.Affine2D().skew_deg(15,15)

    for x in ax.images + ax.lines + ax.collections:
        trans = x.get_transform()
        x.set_transform(r_box+trans) 
        if isinstance(x, PathCollection):
            transoff = x.get_offset_transform()
            x._transOffset = r_box+transoff     
    ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
    ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
    ax.set_aspect('equal', 'box') 
    ax.axis('off')   
#    plt.show()
#    fig.savefig('Figs/' + mouse +'_' + sess+ '/ratemap' + str(it) + '.png', transparent = True, pad_inches = 0.1)
#    fig.savefig('Figs/' + mouse +'_' + sess+ '/ratemap' + str(it) + '.pdf', transparent = True, pad_inches = 0.1)
#    plt.close()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (5,5), dpi = 120)
ax.plot([-0.1,0.6],[-0.1,0.6], c = 'k', ls = ':', alpha  =0.6)
ax.scatter(space3, tor3, s = 30, alpha = 0.75)
ax.scatter(space3[argwhere], tor3[argwhere], c = 'r', marker = 'X', s = 150, lw = 0.1)
ax.set_xlim([-0.1,0.6])
ax.set_ylim([-0.1,0.6])
ax.set_yticklabels('')
ax.set_xticklabels('')
ax.set_yticklabels('')
ax.set_aspect(1/ax.get_data_ratio())
fig.savefig('Figs/' + mouse +'_' + sess+ '/explained_variance.png', transparent = True, pad_inches = 0.1)
fig.savefig('Figs/' + mouse +'_' + sess+ '/explained_variance.pdf', transparent = True, pad_inches = 0.1)



## CEBRA

In [ ]:
from cebra import CEBRA
cebra_time3_model = CEBRA(model_architecture='offset10-model',
                    batch_size=128,
                    learning_rate=0.001,
                    temperature=1.12,
                    output_dimension=3,
                    max_iterations=1000,
                    distance='cosine',
                    conditional='time',
                    device='cuda_if_available',
                    verbose=True,
                    time_offsets=10)

In [ ]:
cebra_time3_model.fit(dim_red_spikes_move_scaled_bu)

In [ ]:
cebra_dec = cebra_time3_model.transform(sspk2)

In [ ]:
cebra_dec = cebra_time3_model.transform(dim_red_spikes_move_scaled_bu)

In [ ]:
fig = plt.figure(figsize = (10,10), dpi = 120)
ax = fig.add_subplot(111,)
ax.scatter(cebra_dec[:,0], cebra_dec[:,1], s = 100, c = hd)
ax.axis('off')
plt.show()

In [ ]:
help(CEBRA)

In [ ]:

plt.hsv()
for ii in [-60,0, 60]:
    for jj in [-90, 0, 90, ]:
        print(ii,jj)
        fig = plt.figure(figsize = (10,10), dpi = 120)
        ax = fig.add_subplot(111, projection = '3d')
        ax.scatter(cebra_dec[:,0], cebra_dec[:,1], cebra_dec[:,2], s = 5, c = hd)
        ax.view_init(ii,jj)
        ax.axis('off')
        plt.show()